In [ ]:
import pandas as pd 
import spotipy
import spotipy.oauth2 as oauth2
from spotipy.oauth2 import SpotifyClientCredentials
import json 
import os
from tqdm import tqdm
import sqlite3

In [ ]:
credentials_file_path = "./credentials.json"

# open the file and load the data into a variable
with open(credentials_file_path, "r") as f:
    credentials = json.load(f)

In [ ]:
def get_info(artist_ids):
     #we will have to loop through the list before calling this function so it applies to each playlist. 
     return [spotify.artist(artist)['genres'] for artist in artist_ids]

In [ ]:
data_files = load_files('./raw_data/playlist')

artist_ids = []
for i in tqdm(data_files): 
    artist_ids.append(get_artist_ids(i))


In [ ]:
list_of_genres = []
for i in tqdm(artist_ids):
    list_of_genres.append(get_info(i))

In [ ]:
new_list = []
for i in list_of_genres:
    new_list.append(list(set([genre for genre_list in i for genre in genre_list])))

In [ ]:
#creates a dictionary with the name of th eplaylist and its id
fixed_dict = {data_names['name']: data_names['uri'] for data_names in data_files}


#getting the followers of all the playlists

number_of_followers = [followers['followers']['total'] for followers in data_files]

In [ ]:

playlist_names = list(fixed_dict.keys())
list_of_uri = list(fixed_dict.values())



playlist_category = dict(zip(playlist_names, new_list))






playlist = list(fixed_dict.keys())

playlist


In [ ]:
#create dataframe playlist name uri and followers

df_playlist = pd.DataFrame({'playlist_name': playlist, 'uri': list_of_uri, 'followers': number_of_followers}).reset_index()

df_playlist = df_playlist.rename(columns={'index': 'id'})
df_playlist['id'] = df_playlist['id'].apply(lambda x: x+1)
df_playlist


In [ ]:
playlist_category = pd.DataFrame({'categories':new_list}).reset_index()
playlist_category = playlist_category.rename(columns = {'index': 'playlist_id'})


playlist_category['playlist_id'] = playlist_category['playlist_id'].apply(lambda x: x+1)

playlist_category = playlist_category.explode('categories')

In [ ]:
playlist_category

In [ ]:
%reload_ext sql
%config SqlMagic.autocommit=True

In [ ]:
%sql sqlite:///data/playlist_uk.db --alias playlist_uk

In [ ]:
DATA_FOLDER = os.path.join('data/')

In [ ]:
conn = sqlite3.connect(os.path.join(DATA_FOLDER, 'playlist_uk.db'))

In [ ]:
%%sql

    -- creating the tables in order for analysis 

DROP TABLE IF EXISTS playlist_name; 

CREATE TABLE playlist_name (
 "id" INTEGER PRIMARY KEY,
 "playlist_name" VARCHAR(50),
 "uri" VARCHAR(50),
 "Followers" INTEGER
);

DROP TABLE IF EXISTS playlist_category; 

CREATE TABLE playlist_category (

    "playlist_id" INTEGER,
    "categories" VARCHAR(50),
    FOREIGN KEY("playlist_id") REFERENCES "playlist_name" ('id')
); 

In [ ]:
#uploading data
df_playlist.to_sql('playlist_name', conn, if_exists='append', index=False)

playlist_category.to_sql('playlist_category', conn, if_exists='append', index=False)